In [ ]:
# default_exp encoding

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
#export
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
import warnings
import copy

def product_moment_corr(x,y):
    '''Product-moment correlation for two ndarrays x, y'''
    from sklearn.preprocessing import StandardScaler
    x = StandardScaler().fit_transform(x)
    y = StandardScaler().fit_transform(y)
    n = x.shape[0]
    r = (1/(n-1))*(x*y).sum(axis=0)
    return r

# Training and validating voxel-wise encoding models
> Functions for training independent Ridge regressions for a large number of voxels and validating their performance

In [ ]:
#export

def get_model_plus_scores(X, y, estimator=None, n_splits=8, scorer=None,
                          voxel_selection=True, validate=True, **kwargs):
    '''Returns multiple estimator trained in a cross-validation on n_splits of the data and scores on the left-out folds

    Parameters

        X : ndarray of shape (samples, features)
        y : ndarray of shape (samples, targets)
        estimator : None or estimator object that implements fit and predict
                    if None, uses RidgeCV per default
        n_splits : int, optional, number of cross-validation splits
        scorer : None or any sci-kit learn compatible scoring function, optional
                 default uses product moment correlation
        voxel_selection : bool, optional, default True
                          Whether to only use voxels with variance larger than zero.
                          This will set scores for these voxels to zero.
        validate : bool, optional, default True
                     Whether to validate the model via cross-validation
                     or to just train the estimator
                     if False, scores will be computed on the training set
        kwargs : additional parameters that will be used to initialize RidgeCV if estimator is None 
    Returns
        tuple of n_splits estimators trained on training folds or single estimator if validation is False
        and scores for all concatenated out-of-fold predictions'''
    from sklearn.utils.estimator_checks import check_regressor_multioutput
    if scorer is None:
        scorer = product_moment_corr
    kfold = KFold(n_splits=n_splits)
    models = []
    score_list = []
    if estimator is None:
        estimator = RidgeCV(**kwargs)
        
    if voxel_selection:
        voxel_var = np.var(y, axis=0)
        y = y[:, voxel_var > 0.]
    if validate:
        for train, test in kfold.split(X, y):
            models.append(copy.deepcopy(estimator).fit(X[train], y[train]))
            if voxel_selection:
                scores = np.zeros_like(voxel_var)
                scores[voxel_var > 0.] =  scorer(y[test], models[-1].predict(X[test]))
            else:
                scores = scorer(y[test], models[-1].predict(X[test]))
            score_list.append(scores[:, None])
        score_list = np.concatenate(score_list, axis=-1)
    else:
        models = estimator.fit(X, y)
        score_list = scorer(y, estimator.predict(X))
    return models, score_list

`get_model_plus_scores` is a convenience function that trains `n_splits` Ridge regressions in a cross-validation scheme and evaluates their performance on the respective test set.

# Examples

First, we create some simulated `stimulus` and `fmri` data.

In [ ]:
stimulus = np.random.randn(1000, 5)
fmri = np.random.randn(1000, 10)

## Using the default Ridge regression

We can now use `get_model_plus_scores` to estimate multiple [RidgeCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html) regressions, one for each voxel (that maps the stimulus representation to this voxel) and one for each split (trained on a different training set and evaluated on the held-out set).
Since sklearn's `RidgeCV` estimator allows multi-output, we get one `RidgeCV` object per split.

In [ ]:
ridges, scores = get_model_plus_scores(stimulus, fmri, n_splits=3)
assert len(ridges) == 3
ridges

[RidgeCV(alphas=array([ 0.1,  1. , 10. ])),
 RidgeCV(alphas=array([ 0.1,  1. , 10. ])),
 RidgeCV(alphas=array([ 0.1,  1. , 10. ]))]

Each `RidgeCV` estimator maps from the feature space to each voxel.
In our example, that means it has 10 (the number of voxels-9 independently trained regression models with 5 coeficients each (the number of features).

In [ ]:
assert ridges[0].coef_.shape == (10, 5)
print(ridges[0].coef_)

[[ 0.01930266  0.0350985  -0.04548384 -0.01058159 -0.07382483]
 [ 0.00183012 -0.00830046 -0.03604675 -0.00016843  0.03161116]
 [-0.04032306  0.01782385  0.02112695  0.01673908 -0.00645515]
 [ 0.0273047  -0.02382577 -0.06169262  0.06232742 -0.03331368]
 [ 0.01294108 -0.04825337 -0.04646228 -0.04701512 -0.00017405]
 [ 0.02008884 -0.07065883  0.01958404 -0.04115758 -0.02967363]
 [ 0.00502653 -0.02164034 -0.00419562 -0.05675778  0.00716245]
 [ 0.0080379   0.03230623  0.01527909 -0.02469508 -0.01681562]
 [ 0.01363082  0.02686557 -0.05923971  0.01392573 -0.00945206]
 [ 0.01665226 -0.01499506 -0.0043113  -0.01658976  0.06103525]]


We also get a set of scores (by default the [product moment correlation](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient), but you can supply your own via the `scorer` argument) that specifies how well we predict left-out data (with the usual caveats of using a correlation coefficient for evaluating it). In our case it is of shape (10, 3) because we predict 10 voxels and use a 3-fold cross-validation, i.e. we split 3 times.

In [ ]:
assert scores.shape == (10, 3)
scores

array([[ 0.11195214,  0.10260332,  0.02153565],
       [-0.06450754, -0.07106461, -0.0447989 ],
       [ 0.00559572, -0.03221425,  0.02866726],
       [-0.04101258, -0.02197306, -0.04277958],
       [ 0.02352969,  0.02008923, -0.02062713],
       [ 0.01027339,  0.03074076,  0.01248573],
       [-0.05974497, -0.03980094, -0.11293944],
       [-0.01607721, -0.02264425, -0.07340733],
       [-0.06009815, -0.05553956,  0.02102434],
       [ 0.02388894, -0.01513094,  0.0904367 ]])

We can also change the parameters of the `RidgeCV` function.
For example, we can use pre-specified hyperparameters, like the values of the regularization parameter $\alpha$ we want to perform a gridsearch over or whether we want to normalize features. If we want to use other parameters for the default `RidgeCV`, we can just pass the parameters as additional keyword arguments:

In [ ]:
alphas = [100]
ridges, scores = get_model_plus_scores(stimulus, fmri, n_splits=3, alphas=alphas,
                                       normalize=True, alpha_per_target=True)
assert ridges[0].normalize
assert ridges[0].alphas.shape == (1,)

AssertionError: 

## Using your own estimator


Additionally, we can use any other estimator that implements `fit` and `predict`.
For example, we can use [CCA](https://scikit-learn.org/stable/modules/generated/sklearn.cross_decomposition.CCA.html) as an encoding model.

In [ ]:
from sklearn import cross_decomposition

our_estimator = cross_decomposition.CCA(n_components=2)

ccas, scores = get_model_plus_scores(stimulus, fmri, our_estimator,
                                       n_splits=3)
assert type(ccas[0]) == cross_decomposition._pls.CCA

If your favorite estimator does not work in the multioutput regime, i.e. it cannot predict multiple targets/voxels, then `get_model_plus_scores` will wrap it into sklearn's [MultiOutputRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputRegressor.html) by default. However, for many voxels this can increase training time by a lot.

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.multioutput import MultiOutputRegressor

our_estimator = MultiOutputRegressor(Lasso())

lassos, scores = get_model_plus_scores(stimulus, fmri, our_estimator,
                                       n_splits=3)
lassos

[MultiOutputRegressor(estimator=Lasso()),
 MultiOutputRegressor(estimator=Lasso()),
 MultiOutputRegressor(estimator=Lasso())]

## Training without validation

We can also train an estimator without any validation, if, for example we want to test on a different dataset. In that case, the scores will be computed with the trained estimator on the training set, i.e. they will contain no information about the generalization performance of the estimator.

In [ ]:
our_estimator = RidgeCV()

model, scores = get_model_plus_scores(stimulus, fmri, our_estimator,
                                       validation=False)
assert type(model) == RidgeCV
assert scores.shape == (10,)